In [1]:
import pandas as pd

# Charger les données
data_files = [
    "./dataset/valeursfoncieres-2022.txt",
    "./dataset/valeursfoncieres-2021.txt",
    "./dataset/valeursfoncieres-2020.txt",
    "./dataset/valeursfoncieres-2019.txt",
    "./dataset/valeursfoncieres-2018.txt"
]

try:
    data = pd.read_pickle('./dataset/valeursfoncieres-2018-2022.pkl')
except:
    dateparse = lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='coerce')
    types = {
        'No voie': 'category',
        'Code postal': 'category',
        'Nature mutation': 'category',
        'Commune': 'category'
    }

    data_frames = []
    for file in data_files:
        data_frames.append(pd.read_csv(file, delimiter='|', parse_dates=['Date mutation'], date_parser=dateparse, dtype=types))

    data = pd.concat(data_frames)
    data.to_pickle('./dataset/valeursfoncieres-2018-2022.pkl')
    
# Définir l'option d'affichage pour afficher toutes les lignes, colonnes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Supprimer les colonnes ayant plus de 7 millions de lignes vides
data = data.dropna(axis=1, thresh=7000000) 

# Supprimer les valeurs nulles de la colonne "Valeurs foncières"
data = data.dropna(subset=['Valeur fonciere', 'Code postal'])

# On convertit la colonne 'Date mutation' en date time
data['Date mutation'] = pd.to_datetime(data['Date mutation'])

# Remplacer les valeurs "2A" par 210 et "2B" par 220 dans la colonne "Code departement"
data['Code departement'] = data['Code departement'].replace({'2A': 210, '2B': 220})

# Changer le type de données de la colonne "Code departement" en int
data['Code departement'] = data['Code departement'].astype(int)

# Changer le type de données de la colonne "Code departement" en int
data['Code postal'] = data['Code postal'].astype(int)

# Convertir les colonnes en int en supprimant les valeurs non valides
columns_to_convert = ['Valeur fonciere', 'Surface reelle bati', 'Nombre pieces principales']
for col in columns_to_convert:
    if col == 'Valeur fonciere' : 
        data[col] = data[col].str.replace(',', '.').fillna(0).astype(float)
    else : 
        data[col] = data[col].fillna(0).astype(float).astype(int)        

# Remplacer les valeurs nulles de la colonne "Surface terrain" par 0 et convertir la colonne en int
data['Surface terrain'] = data['Surface terrain'].fillna(0).astype(int)

#on récupère que les données en ile de france
data = data[data['Code departement'].isin([75, 77, 78, 91, 92, 93, 94, 95])]

#on supprime les valeurs = à 0
data = data[data['Surface reelle bati'] != 0] 

#on supprime les valeurs = à 0
data = data[data['Valeur fonciere'] != 0] 

data['Année mutation'] = data['Date mutation'].dt.year

#on supprime les valeurs = à 0
data = data[data['Surface reelle bati'] != 0] 

#on supprime les valeurs = à 0
data = data[data['Valeur fonciere'] != 0] 

# Écrire le DataFrame dans un fichier CSV
data.to_csv('./dataset/clean_data.csv', index=False)

#on récupère uniquement les données de vente de bien
data_vente_ile_de_france = data[data["Nature mutation"] == "Vente"] 

# Supprimer les colonnes vides
empty_cols = data_vente_ile_de_france.columns[data_vente_ile_de_france.isnull().all()]
data_vente_ile_de_france.drop(empty_cols, axis=1, inplace=True)

# Supprimer les lignes où 'Valeur fonciere' et 'Code postal' sont nuls
data_vente_ile_de_france.dropna(subset=['Valeur fonciere', 'Code postal'], inplace=True)

data_vente_ile_de_france = data_vente_ile_de_france[(data_vente_ile_de_france['Surface reelle bati'] != 0) & (data_vente_ile_de_france['Code type local'].isin([1, 2, 3]))]

# Écrire le DataFrame dans un fichier CSV
data_vente_ile_de_france.to_csv('./dataset/clean_data_vente_idf.csv', index=False)

C:\Users\m.akhmouch\AppData\Local\Temp\ipykernel_17260\4093189787.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_vente_ile_de_france.drop(empty_cols, axis=1, inplace=True)
C:\Users\m.akhmouch\AppData\Local\Temp\ipykernel_17260\4093189787.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_vente_ile_de_france.dropna(subset=['Valeur fonciere', 'Code postal'], inplace=True)
